<a href="https://colab.research.google.com/github/befly098/parallelComputing/blob/main/CUDA_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-10-18 06:34:29--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-10-18 06:34:30--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?-NEB3NVQ7Dt_ddYWeAFGO71ETbkHg6ndAVTJZ38CwxsCUQO2NgIrECjWCDKo1mYwp_8-w4Wk3CPE1OgAO3MjLGYxD4OHuBxN4eitBbKgpXDBm

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4p9u29fd
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4p9u29fd


In [ ]:
%load_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


CUDA 동작 확인 코드 1

In [ ]:
%%cu
#include <iostream>
int main()
{
	std::cout << "Welcome To GeeksforGeeks\n";
	return 0;
}


Welcome To GeeksforGeeks



CUDA 동작 확인 코드 2

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values  
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

CUDA error copying to Host: no CUDA-capable device is detected
result is 0



# **1. hello-cpp.cu**



---



.cu 파일은 C/C++을 기반으로 한 확장자다.
이 확장자를 사용하면 C/C++ 프로그래밍과 CUDA 커널을 호출할 수 있다.

.cu 확장자 파일은 반드시 CUDA 커널을 호출할 필요는 없다.
아래의 코드는 C/C++로만 작성된 코드로, CUDA 커널을 호출하지 않은 예시다

In [ ]:
%%cu

#include <stdio.h>

void hello()
{
    printf("hello");
 
    return;
}

int main()
{
    hello();
 
    return 0;
}

hello


# **2. Hello-cuda.cu**


---



*   \_\_global\_\_


> \_\_global\_\_은 CUDA 함수를 표시하는 키워드다.
이 키워드를 앞에 붙인 함수는 CUDA 커널을 호출한다.


*   <<<x, y>>>


> <<<x, y>>>는 CUDA 함수를 호출할 때 사용할 core 갯수를 지정하는 키워드다.
만약 x = 1, y = 1이라면, 1 * 1 = 1(개)를 사용한다는 뜻이다.


(단, Linux일 경우 아래의 코드를 추가해야 병렬로 실행된다.)


```
#if defined(__linux__)
    cudaDeviceSynchronize();
#endif
    fflush(stdout);
```







In [ ]:
%%cu

#include <stdio.h>

__global__ void hello(void)
{
    printf("hello");

}

int main()
{
    hello<<<1, 1>>>();

#if defined(__linux__)
    cudaDeviceSynchronize();
#endif
    fflush(stdout);
 
    return 0;
}

# **3. Hello-parallel_1.cu**


---



*   함수 병렬로 N번 반복하기


> 만약 사용하는 코어가 <<<1, 8>>> == 8개라면, 어떤 함수를 8번 병렬로 반복한다는 의미다.




*   threadIdx.x


> 몇 번째 코어가 실행 중인지 int 형으로 리턴해주는 함수다.







In [ ]:
%%cu

#include <stdio.h>

__global__ void hello(void)
{
    printf("hello %d \n", threadIdx.x);

}

int main()
{
    hello<<<1, 8>>>();
 
# if defined(__linux__)
    cudaDeviceSynchronize();
# endif
    fflush(stdout);
 
    return 0;
}

# **4. Hello-parallel_2.cu**


---



*   함수 병렬로 X세트, Y번씩 반복하기


> 만약 사용하는 코어가 <<<8, 2>>> == 16개라면, 어떤 함수를 2번 반복하는 과정을 2세트 진행한다는 뜻이다.







In [ ]:
%%cu

#include <stdio.h>

__global__ void hello(void)
{
    printf("hello %d \n", threadIdx.x);

}

int main()
{
    hello<<<8, 2>>>();
 
# if defined(__linux__)
    cudaDeviceSynchronize();
# endif
    fflush(stdout);
 
    return 0;
}

# **5. memcpy.cu**

CUDA를 본격적으로 사용하기 전에, CUDA의 동작 방식을 컴퓨터 내부구조를 통해 알아보도록 하자.<br><br>

*   PC 구조


> PC 구조는 다음과 같다.<br>
(!todo: 그림 추가하기)<br>
north bridge는 CPU, RAM(host memory)을 연결하는 memory controller, south bridge는 보조 메모리와 외부 디바이스를 연결하는 I/O controller다.<br>
CUDA의 동작방식은 north bridge 부분과 관련이 있으므로 north bridge 구조만 살펴보도록 하겠다.<br>
(!todo: 그림 추가하기)<br>
GPU가 설치된 PC는 크게 Host와 Device로 구성되어 있고 PCI가 두 부분을 연결하고 있다. <br>
Host는 우리가 일반적으로 알고 있는 PC의 CPU와 main memory로 구성되어 있다. Main memory는 RAM 혹은 host memory라고 칭하기도 한다.<br>
Device는 그래픽 카드의 GPU, video memory로 구성된다. video memory도 VRAM 혹은 device memory라는 별칭으로 불린다.<br>



*   GPU가 코드를 실행하는 방법
*   



In [ ]:
%%cu

#include <stdio.h>

int main()
{
    const int SIZE = 9;
    const float a[SIZE] = {1., 2., 3., 4., 5., 6., 7., 8., 9.};
    float b[SIZE] = {0., 0., 0., 0., 0., 0., 0., 0., 0.};
 
    printf("<before memcpy>\n");
    printf("a = {%f, %f, %f, %f, %f, %f, %f, %f, %f}\n", a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8]);
    printf("b = {%f, %f, %f, %f, %f, %f, %f, %f, %f}\n\n", b[0], b[1], b[2], b[3], b[4], b[5], b[6], b[7], b[8]);
 
    fflush(stdout);
 
    float* dev_a = nullptr;
    float* dev_b = nullptr;
 
    cudaMalloc((void**)&dev_a, SIZE * sizeof(float));
    cudaMalloc((void**)&dev_b, SIZE * sizeof(float));
 
    cudaMemcpy(dev_a, a, SIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, dev_a, SIZE * sizeof(float), cudaMemcpyDeviceToDevice);
    cudaMemcpy(b, dev_b, SIZE * sizeof(float), cudaMemcpyDeviceToHost);
 
    cudaFree(dev_a);
    cudaFree(dev_b);
 
    printf("<after memcpy>\n");
    printf("a = {%f, %f, %f, %f, %f, %f, %f, %f, %f}\n", a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8]);
    printf("b = {%f, %f, %f, %f, %f, %f, %f, %f, %f}\n\n", b[0], b[1], b[2], b[3], b[4], b[5], b[6], b[7], b[8]);
 
# if defined(__linux__)
    cudaDeviceSynchronize();
# endif
    fflush(stdout);
 
    return 0;
}

<before memcpy>
a = {1.000000, 2.000000, 3.000000, 4.000000, 5.000000, 6.000000, 7.000000, 8.000000, 9.000000}
b = {0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000}

<after memcpy>
a = {1.000000, 2.000000, 3.000000, 4.000000, 5.000000, 6.000000, 7.000000, 8.000000, 9.000000}
b = {0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000}


